Import some packages. We use [networkx](https://networkx.github.io/documentation/stable/index.html) for graphs in Python 

In [ ]:
import numpy as np
import networkx as nx
import random 
import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams['figure.figsize'] = (12, 12) # set default size of plots
plt.rcParams.update({'font.size': 18}) # set default font size
plt.rcParams['image.cmap']='hot' # set color map

Now define some functions that build our graph. These are adapted from [here](https://networkx.github.io/documentation/stable/_modules/networkx/generators/random_graphs.html#barabasi_albert_graph). Don't worry if you can't follow the details.

In [ ]:
def _random_subset(seq,m):
    """ Return m unique elements from seq.

    This differs from random.sample which can return repeated
    elements if seq holds repeated elements.
    """
    targets=set()
    while len(targets)<m:
        x=random.choice(seq)
        targets.add(x)
    return targets

In [ ]:
def dms_graph_basic(N,m,k0,seed=None):
    '''
    Build DorogovtsevMendesSamukhin graph. 
    Graph is 'basic' as we use the case m=m0 (as required for problem sheet m=m0=5).
    
    Usage: G=dms_graph_basic(N,m,k0,seed=None)
    
    Input:
    N - Number of nodes
    m - see problem sheet (=m0 also)
    k0 - damping factor
    
    Output:
    G - A Networkx graph 
    '''

    if seed is not None:
        random.seed(seed)

    G=nx.complete_graph(m)
    
    targets=range(m)
    
    repeated_nodes=range(m)*(m-1+k0)

    source=m
    
    while source<N:
        G.add_edges_from(zip([source]*m,targets))
        repeated_nodes.extend(targets)
        repeated_nodes.extend([source]*(m+k0))
        targets = _random_subset(repeated_nodes,m)
        source += 1
    return G

**Part a)** Here we study the degree distribution. Lets calculate for 1 realization..

In [ ]:
###### Parameters ########

k0=0

N=1000

#########################

G=dms_graph_basic(N,5,k0) # build graph

degs=nx.degree_histogram(G) # get degree histogram (list)

if len(degs)<N: # standadize length (for when we run many realizations)
    degs=degs+[0]*(N-len(degs))
degs=np.asarray(degs,dtype=float) # make numpy array

degs=degs/np.sum(degs) # normalize

And plot! Note I use a power of $$-3-\frac{k_0}{m}$$ I think it depends on how you interpret things..

In [ ]:
x=np.asarray(range(N),dtype=float)

plt.plot(x,degs,'xr',mew=3,ms=8,label='Empirical')

pwr=-3.0-float(k0)/5.0
plt.plot(x,100*x**pwr,'k-',lw=3,label='$=100k^{-3-k_0/m}$')

plt.xscale('log')
plt.yscale('log')

plt.xlabel('$k$')
plt.ylabel('$p(k)$')
plt.legend()
plt.title('Degree distribution, 1 realization, $k_0=${}'.format(k0))

plt.ylim(1.0/(2*N),0.5)
plt.xlim(4,200)

**Now average over 20 realizations!** (to do)

**Part b)** Now we compute $k_{nn}(k)$. What is the expected degree of my neighbour, given I have degree $k$? Here we calculate for 1 realization..

In [ ]:
################

k0=0

N=1000

kmax=200 # the max degree we're interested in (high degrees are rare)

################

Knn=np.zeros(kmax) 

G=dms_graph_basic(N,5,k0) # build graph

A=nx.to_numpy_matrix(G) # get adj matix as numpy array

d=nx.degree(G).values()
d=np.asarray(d,dtype=float) # get degrees, convert to numpy, reshape.  
d=np.reshape(d,(1,N)) # So d[i] is degree of node i

for k in range(kmax): # loop on k
    delta=np.equal(k,d).astype(int)
    delta=np.reshape(delta,(1,N)) # delta vector. delta[i]=1 if d[i]=k.

    # numerator (uses broadcasting. Google me!)
    t1=np.multiply(np.multiply(np.multiply(A,delta.T),d),1/d.T) 
    t1=np.sum(np.sum(t1))

    t2=np.sum(delta) # denominator

    if t2!=0: # fill in Knn[k]
        Knn[k]=t1/t2


Now plot!

In [ ]:
plt.plot(range(kmax),Knn,'rx',ms=8,mew=3)

plt.xlim(0,160)
plt.ylim(15,25)

plt.xlabel('$k$')
plt.ylabel('$k_{nn}(k)$')
plt.title('$k_0=${}, "averaged" over 1 realization.'.format(k0))

**Now average over 20 realizations** (to do)

**Part c)** Now look at Wigner semi-circle law <img src="https://i.imgur.com/qH0V3FZ.png" width="600px"\>

In [ ]:
from scipy import stats # you need this package

################

k0=0

N=1000

################

G=dms_graph_basic(N,5,k0)

A=nx.to_numpy_matrix(G)
evals,_=np.linalg.eig(A/np.sqrt(N))
empiricalPDF_func=stats.gaussian_kde(evals,bw_method=0.05)      # returns a function

x=np.linspace(evals.min(),evals.max(),1000) # empirical specral density
plt.plot(x,empiricalPDF_func(x),'b-',lw=3)

# PLOT THE WIGNER CIRCLE!!

plt.xlim(0.3,-0.3)
plt.ylabel(r'$\rho(\lambda)$')
plt.xlabel(r'$\lambda$')

**Add the Wigner circle to the plot** (to do)